In [1]:
import requests
import threading
from requests import Session
from queue import Queue
import csv
import time

class GitHub_API_Task:
    def __init__(self):
        self.session = Session()
        self.details_of_user = []
        self.lock = threading.Lock()

    def fetch_user_details(self, username):
        url = f"https://api.github.com/users/{username}"
        response = self.session.get(url)
        if response.status_code == 200:
            user_data = response.json()
            repositories = self.fetch_user_repositories(username)
            user_data['repositories'] = repositories
            with self.lock:
                self.details_of_user.append(user_data)

    def fetch_user_repositories(self, username):
        url = f"https://api.github.com/users/{username}/repos"
        response = self.session.get(url)
        if response.status_code == 200:
            repositories_data = response.json()
            repositories = [repo['name'] for repo in repositories_data]
            return repositories
        return []

    def saving_the_user_details(self, filename):
        with open(filename, 'w', newline='') as csvfile:
            fieldnames = ['Username', 'Name', 'Location', 'Email', 'Public Repos', 'Repositories']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for user_data in self.details_of_user:
                writer.writerow({
                    'Username': user_data['login'],
                    'Name': user_data['name'],
                    'Location': user_data['location'],
                    'Email': user_data['email'],
                    'Public Repos': user_data['public_repos'],
                    'Repositories': ', '.join(user_data['repositories'])
                })

def user_details_worker(usernames_queue, User):
    while True:
        username = usernames_queue.get()
        if username is None:
            break
        User.fetch_user_details(username)
        usernames_queue.task_done()

# Create an instance of the Class
User = GitHub_API_Task()

# Create a queue to hold the usernames
usernames_queue = Queue()

# Create and start multiple producer threads
num_threads = 10
threads = []
for i in range(num_threads):
    thread = threading.Thread(target=user_details_worker, args=(usernames_queue, User))
    thread.start()
    threads.append(thread)

# Enqueue the usernames
usernames = ["eugeneyan", "DataTalksClub", "airbytehq", "great-expectations", "growthbook", "cloudquery",
             "whoiskatrin", "aws", "treeverse", "adilkhash"]
for username in usernames:
    usernames_queue.put(username)

# Wait for all tasks to be completed
usernames_queue.join()

# Stop the worker threads
for j in range(num_threads):
    usernames_queue.put(None)
for thread in threads:
    thread.join()

# Save the user details to a CSV file
print("Number of user details:", len(User.details_of_user))
User.saving_the_user_details("GitHub.csv")


Number of user details: 10


<font size =20><b>Breakdown of the code workflow:</b></font>

1. Import necessary libraries: The code begins by importing the required libraries, including `requests`, `threading`, `Session` from `requests`, `Queue` from `queue`, and `csv`.

2. Define the `GitHub_API_Task` class: The class initializes a session, an empty list to store user details, and a lock to synchronize access to the shared data.

3. `fetch_user_details` method: This method takes a username as input and fetches the user details from the GitHub API using an HTTP GET request. It also calls the `fetch_user_repositories` method to get the user's repositories and adds the repositories to the user data. The user data is then appended to the `details_of_user` list in a thread-safe manner using a lock.

4. `fetch_user_repositories` method: This method fetches the repositories of a given username from the GitHub API using an HTTP GET request. It extracts the repository names from the API response and returns a list of repository names.

5. `saving_the_user_details` method: This method saves the user details to a CSV file. It opens the file in write mode and writes the field names as the header. It then iterates over the user data in `details_of_user` and writes each user's details as a row in the CSV file.

6. `user_details_worker` function: This function is the target function for the worker threads. It continuously retrieves usernames from the `usernames_queue`, calls the `fetch_user_details` method to fetch the user details, and marks the task as done using `task_done()`. The function exits when it encounters a `None` value in the queue.

7. Create an instance of the `GitHub_API_Task` class: An instance named `User` is created.

8. Create a queue for usernames: The code creates a `Queue` named `usernames_queue` to hold the usernames.

9. Create and start multiple producer threads: A loop is used to create a specified number of worker threads (`num_threads`). Each thread is given the `user_details_worker` function as the target and provided with the `usernames_queue` and `User` instance as arguments. The threads are then started and stored in a list.

10. Enqueue the usernames: The code enqueues a list of usernames into the `usernames_queue`.

11. Wait for all tasks to be completed: The code calls `join()` on the `usernames_queue` to block until all tasks in the queue are marked as done. This ensures that all user details are fetched before proceeding.

12. Stop the worker threads: The code puts `None` into the `usernames_queue` for each worker thread and joins all the threads. This signals the worker threads to exit once they finish processing their current task.

13. Save the user details to a CSV file: Finally, the `saving_the_user_details` method is called to save the fetched user details to a CSV file named "GitHub.csv".

That's the overall workflow of the code. It utilizes multiple threads to concurrently fetch user details from the GitHub API and ensures the data is processed and saved correctly.